#### VACCINATION DATA ANALYSIS PYTHON SCRIPT

##### COVERAGE DATA

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import re

def clean_and_normalize_data(df):
    """
    Clean and normalize the dataset with improved numeric handling
    """
    # Remove duplicates
    df = df.drop_duplicates()
    
    # Standardize text columns
    text_columns = ['GROUP', 'CODE', 'NAME', 'ANTIGEN', 'ANTIGEN_DESCRIPTION', 
                   'COVERAGE_CATEGORY', 'COVERAGE_CATEGORY_DESCRIPTION']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
    
    # Clean YEAR column
    if 'YEAR' in df.columns:
        df['YEAR'] = pd.to_numeric(
            df['YEAR'].astype(str).str.replace('[^0-9]', '', regex=True), 
            errors='coerce'
        ).fillna(0).astype(int)
    
    # Clean TARGET_NUMBER - convert to bigint equivalent
    if 'TARGET_NUMBER' in df.columns:
        df['TARGET_NUMBER'] = pd.to_numeric(
            df['TARGET_NUMBER'].astype(str).str.replace('[^0-9]', '', regex=True), 
            errors='coerce'
        ).fillna(0)
        # Cap at MySQL BIGINT maximum if needed
        df['TARGET_NUMBER'] = df['TARGET_NUMBER'].clip(lower=-9223372036854775808, upper=9223372036854775807)
    
    # Clean DOSES - similar to TARGET_NUMBER
    if 'DOSES' in df.columns:
        df['DOSES'] = pd.to_numeric(
            df['DOSES'].astype(str).str.replace('[^0-9]', '', regex=True), 
            errors='coerce'
        ).fillna(0)
        df['DOSES'] = df['DOSES'].clip(lower=-9223372036854775808, upper=9223372036854775807)
    
    # Clean COVERAGE (percentage)
    if 'COVERAGE' in df.columns:
        df['COVERAGE'] = pd.to_numeric(
            df['COVERAGE'].astype(str).str.replace('[^0-9.]', '', regex=True), 
            errors='coerce'
        ).fillna(0)
        df['COVERAGE'] = df['COVERAGE'].clip(0, 100).round(2)
    
    # Fill NA/NaN values
    df[text_columns] = df[text_columns].fillna('UNKNOWN')
    
    return df

def create_mysql_connection(host_name, user_name, user_password, db_name):
    """
    Create MySQL database connection
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def load_data_to_mysql(df, connection):
    """
    Load cleaned data into MySQL table
    """
    cursor = connection.cursor()
    
    # Prepare data for insertion
    data_to_insert = []
    for _, row in df.iterrows():
        data_to_insert.append(tuple(row))
    
    # SQL query for inserting data
    insert_query = """
    INSERT INTO coverage_data (
        `GROUP`, `CODE`, `NAME`, `YEAR`, `ANTIGEN`, `ANTIGEN_DESCRIPTION`, 
        `COVERAGE_CATEGORY`, `COVERAGE_CATEGORY_DESCRIPTION`, 
        `TARGET_NUMBER`, `DOSES`, `COVERAGE`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    try:
        cursor.executemany(insert_query, data_to_insert)
        connection.commit()
        print(f"Successfully inserted {len(data_to_insert)} records into MySQL")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close()

def process_excel_to_mysql(excel_file_path, host, user, password, database):
    """
    Main function to process Excel file and load to MySQL
    """
    # Read Excel file
    try:
        df = pd.read_excel(excel_file_path)
        print("Excel file read successfully")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return
    
    # Clean and normalize data
    df_clean = clean_and_normalize_data(df)
    
    # Create MySQL connection
    connection = create_mysql_connection(host, user, password, database)
    if connection:
        load_data_to_mysql(df_clean, connection)
        connection.close()

# Example usage
if __name__ == "__main__":
    # Configuration - replace with your actual details
    excel_file_path = 'C:\\Users\\priya\\Downloads\\VACCINE\\Vaccination project\\coverage-data.xlsx'  # Update with your Excel file path
    mysql_host = 'localhost'
    mysql_user = 'root'
    mysql_password = 'AAbb@@11'
    mysql_database = 'vaccination_analysis'
    
    process_excel_to_mysql(
        excel_file_path, 
        mysql_host, 
        mysql_user, 
        mysql_password, 
        mysql_database
    )

Excel file read successfully
MySQL Database connection successful
Successfully inserted 399859 records into MySQL


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def clean_and_normalize_disease_data(df):
    """
    Clean and normalize the disease dataset
    """
    # Remove duplicates
    df = df.drop_duplicates()
    
    # Standardize text columns
    text_columns = ['GROUP', 'CODE', 'NAME', 'DISEASE', 'DISEASE_DESCRIPTION', 'DENOMINATOR']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
    
    # Clean YEAR column
    if 'YEAR' in df.columns:
        df['YEAR'] = pd.to_numeric(
            df['YEAR'].astype(str).str.replace('[^0-9]', '', regex=True), 
            errors='coerce'
        ).fillna(0).astype(int)
    
    # Clean INCIDENCE_RATE (handling decimal values)
    if 'INCIDENCE_RATE' in df.columns:
        df['INCIDENCE_RATE'] = pd.to_numeric(
            df['INCIDENCE_RATE'].astype(str).str.replace('[^0-9.]', '', regex=True), 
            errors='coerce'
        ).fillna(0)
        # No negative incidence rates
        df['INCIDENCE_RATE'] = df['INCIDENCE_RATE'].clip(lower=0)
    
    # Fill NA/NaN values
    df[text_columns] = df[text_columns].fillna('UNKNOWN')
    
    return df

def create_disease_mysql_table(connection):
    """
    Create the disease_data table in MySQL
    """
    cursor = connection.cursor()
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS disease_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        `GROUP` VARCHAR(50) NOT NULL,
        `CODE` VARCHAR(10) NOT NULL,
        `NAME` VARCHAR(100) NOT NULL,
        `YEAR` INT NOT NULL,
        `DISEASE` VARCHAR(50) NOT NULL,
        `DISEASE_DESCRIPTION` VARCHAR(255),
        `DENOMINATOR` VARCHAR(50) NOT NULL,
        `INCIDENCE_RATE` DECIMAL(10,4),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        UNIQUE KEY unique_disease_record (
            `GROUP`, `CODE`, `NAME`, `YEAR`, `DISEASE`, `DENOMINATOR`
        )
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """
    
    try:
        cursor.execute(create_table_query)
        connection.commit()
        print("Disease data table created successfully")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close()

def load_disease_data_to_mysql(df, connection):
    """
    Load cleaned disease data into MySQL table
    """
    cursor = connection.cursor()
    
    # Prepare data for insertion
    data_to_insert = []
    for _, row in df.iterrows():
        data_to_insert.append(tuple(row))
    
    # SQL query for inserting data
    insert_query = """
    INSERT INTO disease_data (
        `GROUP`, `CODE`, `NAME`, `YEAR`, `DISEASE`, `DISEASE_DESCRIPTION`, 
        `DENOMINATOR`, `INCIDENCE_RATE`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    try:
        cursor.executemany(insert_query, data_to_insert)
        connection.commit()
        print(f"Successfully inserted {len(data_to_insert)} records into disease_data table")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close()

def process_disease_excel_to_mysql(excel_file_path, host, user, password, database):
    """
    Main function to process disease Excel file and load to MySQL
    """
    # Read Excel file
    try:
        df = pd.read_excel(excel_file_path)
        print("Disease Excel file read successfully")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return
    
    # Clean and normalize data
    df_clean = clean_and_normalize_disease_data(df)
    
    # Create MySQL connection
    connection = create_mysql_connection(host, user, password, database)
    if connection:
        # Create table if not exists
        create_disease_mysql_table(connection)
        
        # Load data
        load_disease_data_to_mysql(df_clean, connection)
        connection.close()

# Example usage
if __name__ == "__main__":
    # Configuration - replace with your actual details
    excel_file_path = 'C:\\Users\\priya\\Downloads\\VACCINE\\Vaccination project\\incidence-rate-data.xlsx'  # Update with your Excel file path
    mysql_host = 'localhost'
    mysql_user = 'root'
    mysql_password = 'AAbb@@11'
    mysql_database = 'vaccination_analysis'
    
    process_disease_excel_to_mysql(
        excel_file_path, 
        mysql_host, 
        mysql_user, 
        mysql_password, 
        mysql_database
    )

Disease Excel file read successfully
MySQL Database connection successful
Disease data table created successfully
Successfully inserted 84946 records into disease_data table


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def clean_and_normalize_case_data(df):
    """
    Clean and normalize the case dataset
    """
    # Remove duplicates
    df = df.drop_duplicates()
    
    # Standardize text columns
    text_columns = ['GROUP', 'CODE', 'NAME', 'DISEASE', 'DISEASE_DESCRIPTION']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
    
    # Clean YEAR column
    if 'YEAR' in df.columns:
        df['YEAR'] = pd.to_numeric(
            df['YEAR'].astype(str).str.replace('[^0-9]', '', regex=True), 
            errors='coerce'
        ).fillna(0).astype(int)
    
    # Clean CASES column (handle empty strings and convert to integer)
    if 'CASES' in df.columns:
        df['CASES'] = pd.to_numeric(
            df['CASES'].astype(str).str.replace('[^0-9]', '', regex=True), 
            errors='coerce'
        ).fillna(0).astype(int)
        # No negative case counts
        df['CASES'] = df['CASES'].clip(lower=0)
    
    # Fill NA/NaN values
    df[text_columns] = df[text_columns].fillna('UNKNOWN')
    
    return df

def create_case_mysql_table(connection):
    """
    Create the case_data table in MySQL
    """
    cursor = connection.cursor()
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS case_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        `GROUP` VARCHAR(50) NOT NULL,
        `CODE` VARCHAR(10) NOT NULL,
        `NAME` VARCHAR(100) NOT NULL,
        `YEAR` INT NOT NULL,
        `DISEASE` VARCHAR(50) NOT NULL,
        `DISEASE_DESCRIPTION` VARCHAR(255),
        `CASES` INT UNSIGNED NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        UNIQUE KEY unique_case_record (
            `GROUP`, `CODE`, `NAME`, `YEAR`, `DISEASE`
        )
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """
    
    try:
        cursor.execute(create_table_query)
        connection.commit()
        print("Case data table created successfully")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close()

def load_case_data_to_mysql(df, connection):
    """
    Load cleaned case data into MySQL table
    """
    cursor = connection.cursor()
    
    # Prepare data for insertion
    data_to_insert = []
    for _, row in df.iterrows():
        data_to_insert.append(tuple(row))
    
    # SQL query for inserting data
    insert_query = """
    INSERT INTO case_data (
        `GROUP`, `CODE`, `NAME`, `YEAR`, `DISEASE`, `DISEASE_DESCRIPTION`, `CASES`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        `DISEASE_DESCRIPTION` = VALUES(`DISEASE_DESCRIPTION`),
        `CASES` = VALUES(`CASES`),
        `updated_at` = CURRENT_TIMESTAMP
    """
    
    try:
        cursor.executemany(insert_query, data_to_insert)
        connection.commit()
        print(f"Successfully inserted/updated {len(data_to_insert)} records into case_data table")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close()

def process_case_excel_to_mysql(excel_file_path, host, user, password, database):
    """
    Main function to process case Excel file and load to MySQL
    """
    # Read Excel file
    try:
        df = pd.read_excel(excel_file_path)
        print("Case Excel file read successfully")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return
    
    # Clean and normalize data
    df_clean = clean_and_normalize_case_data(df)
    
    # Create MySQL connection
    connection = create_mysql_connection(host, user, password, database)
    if connection:
        # Create table if not exists
        create_case_mysql_table(connection)
        
        # Load data
        load_case_data_to_mysql(df_clean, connection)
        connection.close()

# Example usage
if __name__ == "__main__":
    # Configuration - replace with your actual details
    excel_file_path = 'C:\\Users\\priya\\Downloads\\VACCINE\\Vaccination project\\reported-cases-data.xlsx'  # Update with your Excel file path
    mysql_host = 'localhost'
    mysql_user = 'root'
    mysql_password = 'AAbb@@11'
    mysql_database = 'vaccination_analysis'
    
    process_case_excel_to_mysql(
        excel_file_path, 
        mysql_host, 
        mysql_user, 
        mysql_password, 
        mysql_database
    )

Case Excel file read successfully
MySQL Database connection successful
Case data table created successfully
Successfully inserted/updated 84870 records into case_data table


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def create_mysql_connection(host_name, user_name, user_password, db_name):
    """Create MySQL database connection"""
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
        return connection
    except Error as err:
        print(f"Error: '{err}'")
        return None

def clean_and_normalize_vaccine_data(df):
    """
    Clean and normalize the vaccine introduction dataset with robust INTRO handling
    """
    # Remove duplicates
    df = df.drop_duplicates()
    
    # Standardize text columns with length enforcement
    text_columns = ['ISO_3_CODE', 'COUNTRYNAME', 'WHO_REGION', 'DESCRIPTION']
    for col in text_columns:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
                .str.upper()
            )
    
    # Enforce maximum lengths
    df['ISO_3_CODE'] = df['ISO_3_CODE'].str[:3]
    df['WHO_REGION'] = df['WHO_REGION'].str[:10]
    df['COUNTRYNAME'] = df['COUNTRYNAME'].str[:100]
    df['DESCRIPTION'] = df['DESCRIPTION'].str[:255]
    
    # Clean YEAR column
    if 'YEAR' in df.columns:
        df['YEAR'] = (
            pd.to_numeric(
                df['YEAR'].astype(str).str.replace('[^0-9]', '', regex=True),
                errors='coerce'
            )
            .fillna(0)
            .astype(int)
        )
    
    # Clean INTRO column - advanced handling for complex values
    if 'INTRO' in df.columns:
        # First extract the main yes/no indicator
        df['INTRO'] = (
            df['INTRO']
            .astype(str)
            .str.strip()
            .str.upper()
            .str.extract(r'^(YES|Y|TRUE|T|1|NO|N|FALSE|F|0)', expand=False)
        )
        
        # Then map to binary values
        intro_map = {
            'YES': 1, 'Y': 1, 'TRUE': 1, 'T': 1, '1': 1,
            'NO': 0, 'N': 0, 'FALSE': 0, 'F': 0, '0': 0
        }
        df['INTRO'] = (
            df['INTRO']
            .map(intro_map)
            .fillna(0)  # Treat any non-matching values as 0 (NO)
            .astype(int)
        )
    
    # Fill remaining NA/NaN values
    df[text_columns] = df[text_columns].fillna('UNKNOWN')
    
    return df

def load_vaccine_data_to_mysql(df, connection):
    """
    Load cleaned vaccine data into MySQL table
    """
    cursor = connection.cursor()
    
    # Prepare data for insertion
    data_to_insert = []
    for _, row in df.iterrows():
        data_to_insert.append((
            row['ISO_3_CODE'],
            row['COUNTRYNAME'],
            row['WHO_REGION'],
            row['YEAR'],
            row['DESCRIPTION'],
            row['INTRO']
        ))
    
    # SQL query for inserting data
    insert_query = """
    INSERT INTO vaccine_intro (
        `ISO_3_CODE`, `COUNTRYNAME`, `WHO_REGION`, `YEAR`, 
        `DESCRIPTION`, `INTRO`
    ) VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        `WHO_REGION` = VALUES(`WHO_REGION`),
        `INTRO` = VALUES(`INTRO`),
        `DESCRIPTION` = VALUES(`DESCRIPTION`)
    """
    
    try:
        cursor.executemany(insert_query, data_to_insert)
        connection.commit()
        print(f"Successfully inserted/updated {len(data_to_insert)} records")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close()

def process_vaccine_excel_to_mysql(excel_file_path, host, user, password, database):
    """
    Main function to process vaccine Excel file and load to MySQL
    """
    # Read Excel file
    try:
        df = pd.read_excel(excel_file_path)
        print("Excel file read successfully")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return
    
    # Clean and normalize data
    df_clean = clean_and_normalize_vaccine_data(df)
    
    # Create MySQL connection and load data
    connection = create_mysql_connection(host, user, password, database)
    if connection:
        try:
            load_vaccine_data_to_mysql(df_clean, connection)
        finally:
            connection.close()

if __name__ == "__main__":
    # Configuration - replace with your actual details
    excel_file_path = 'C:\\Users\\priya\\Downloads\\VACCINE\\Vaccination project\\vaccine-introduction-data.xlsx'  # Update with your Excel file path
    mysql_host = 'localhost'
    mysql_user = 'root'
    mysql_password = 'AAbb@@11'
    mysql_database = 'vaccination_analysis'
    
    process_vaccine_excel_to_mysql(
        excel_file_path, 
        mysql_host, 
        mysql_user, 
        mysql_password, 
        mysql_database
    )




Excel file read successfully
MySQL Database connection successful
Successfully inserted/updated 138321 records
